In [600]:
import math
import numpy as np
import pandas as pd
import yfinance as yf
from scipy import optimize

In [601]:
market = '^GSPC'

tickers = [
    'AAPL',
    'MSFT',
    'GOOG',
    'AMZN',
    'BRK-B',
    'V',
    'XOM',
    'UNH',
    'JNJ',
    'NVDA',
    market
]

period = '5y'      # Options are 1d, 5d, 1mo, 3mo, 6mo, 1y, 2y, 5y, 10y, ytd, and max

interval = '1d'    # Options are 1m, 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk, 1mo, and 3mo

allow_short = False

r_f = 0.01         # Risk-free rate

In [602]:
fields = []
for i in range(len(tickers)):
    fields.append((tickers[i], 'Close'))

df = yf.download(
    tickers = tickers,
    period = period,
    interval = interval,
    group_by = 'ticker'
)[fields]

[*********************100%***********************]  11 of 11 completed


In [603]:
norm_returns = df.pct_change().dropna()
log_returns = np.log(1 + df.pct_change().dropna())

norm_cov = norm_returns.cov() * 252
log_cov = log_returns.cov() * 252

In [604]:
norm_returns = norm_returns.mean() * 252
log_returns = log_returns.mean() * 252

In [605]:
def find_beta(cov, stock, market):
    a = cov[(stock, 'Close')][(market, 'Close')]
    b = cov[(market, 'Close')][(market, 'Close')]
    return (a / b)

betas = pd.DataFrame(columns = ['Beta'])
for i in tickers:
    betas.loc[i] = find_beta(norm_cov, i, market)

In [606]:
capm_returns = pd.DataFrame()
capm_returns['Returns'] = norm_returns - r_f
for i in range(len(tickers)):
    capm_returns.iloc[i]['Returns'] = capm_returns.iloc[i]['Returns'] * betas.iloc[i] + r_f

In [607]:
def portfolio_var(w, cov):
    w = np.matrix(w)
    cov = np.matrix(cov)
    result = w * cov * w.T
    return result

def portfolio_returns(w, yearly_returns):
    w = np.matrix(w)
    yearly_returns = np.matrix(yearly_returns)
    return np.sum(w * yearly_returns.T)

def portfolio_sharpe(w, cov, r_f):
    a = portfolio_returns(w, cov) - r_f
    b = math.sqrt(portfolio_var(w, cov))
    return -a / b

In [608]:
w_cons = {'type': 'eq', 'fun': lambda x: np.sum(x) - 1}

bounds = [()]
if allow_short:
    bounds = tuple((-np.inf, np.inf) for i in tickers)
else:
    bounds = tuple((0, 1) for i in tickers)

w0 = [1 / len(tickers)] * len(tickers)

In [609]:
dist = 'log'     # 'norm' or 'log'

cov_choice = norm_cov
returns_choice = norm_returns
if dist == 'log':
    cov_choice = log_cov
    returns_choice = log_returns

In [610]:
abs_min_var = optimize.minimize(
    fun = portfolio_var,
    x0 = w0,
    args = cov_choice,
    method = 'SLSQP',
    bounds = bounds,
    constraints = w_cons
)

print('Absolute Minimum Variance Portfolio')
for i in range(len(tickers)):
    print(tickers[i], '=', round(abs_min_var.x[i] * 100, 3), '%')
print('Expected Annual Returns =', round(portfolio_returns(abs_min_var.x, returns_choice) * 100, 3), '%')
print('CAPM Expected Annual Returns =', round(portfolio_returns(abs_min_var.x, capm_returns['Returns']) * 100, 3), '%')
print('Sharpe Ratio =', round(-portfolio_sharpe(abs_min_var.x, cov_choice, r_f), 3))

Absolute Minimum Variance Portfolio
AAPL = 0.0 %
MSFT = 0.0 %
GOOG = 0.0 %
AMZN = 6.461 %
BRK-B = 18.303 %
V = 0.0 %
XOM = 4.438 %
UNH = 0.0 %
JNJ = 55.286 %
NVDA = 0.0 %
^GSPC = 15.512 %
Expected Annual Returns = 5.466 %
CAPM Expected Annual Returns = 6.877 %
Sharpe Ratio = 2.192


In [611]:
cons_choice = 'returns'     # 'returns' or 'beta' (can only constrain one at a time)

returns_target = 0.12
beta_target = 1.2

return_cons = {'type': 'eq', 'fun': lambda x: portfolio_returns(x, returns_choice) - returns_target}
beta_cons = {'type': 'eq', 'fun': lambda x: portfolio_returns(x, betas['Beta']) - beta_target}

min_var = optimize.minimize(
    fun = portfolio_var,
    x0 = w0,
    args = cov_choice,
    method = 'SLSQP',
    bounds = bounds,
    constraints = (w_cons, return_cons) if cons_choice == 'returns' else (w_cons, beta_cons)
)

print('Minimum Variance Portfolio with Beta/Returns Constraint')
for i in range(len(tickers)):
    print(tickers[i], '=', round(min_var.x[i] * 100, 3), '%')
print('Expected Annual Returns =', round(portfolio_returns(min_var.x, returns_choice) * 100, 3), '%')
print('CAPM Expected Annual Returns =', round(portfolio_returns(min_var.x, capm_returns['Returns']) * 100, 3), '%')
print('Sharpe Ratio =', round(-portfolio_sharpe(min_var.x, cov_choice, r_f), 3))

Minimum Variance Portfolio with Beta/Returns Constraint
AAPL = 22.115 %
MSFT = 7.316 %
GOOG = 0.0 %
AMZN = 0.0 %
BRK-B = 16.756 %
V = 0.0 %
XOM = 3.352 %
UNH = 12.388 %
JNJ = 38.073 %
NVDA = 0.0 %
^GSPC = 0.0 %
Expected Annual Returns = 12.0 %
CAPM Expected Annual Returns = 16.293 %
Sharpe Ratio = 2.4


In [612]:
sharpe_ratio = optimize.minimize(
    fun = portfolio_sharpe,
    x0 = w0,
    args = (cov_choice, r_f),
    method = 'SLSQP',
    bounds = bounds,
    constraints = w_cons
)

print('Maximum Sharpe Ratio Portfolio')
for i in range(len(tickers)):
    print(tickers[i], '=', round(sharpe_ratio.x[i] * 100, 3), '%')
print('Expected Annual Returns =', round(portfolio_returns(sharpe_ratio.x, returns_choice) * 100, 3), '%')
print('CAPM Expected Annual Returns =', round(portfolio_returns(sharpe_ratio.x, capm_returns['Returns']) * 100, 3), '%')
print('Sharpe Ratio =', round(-portfolio_sharpe(sharpe_ratio.x, cov_choice, r_f), 3))

Maximum Sharpe Ratio Portfolio
AAPL = 9.612 %
MSFT = 9.935 %
GOOG = 9.288 %
AMZN = 9.053 %
BRK-B = 9.231 %
V = 9.576 %
XOM = 9.432 %
UNH = 9.509 %
JNJ = 8.091 %
NVDA = 9.729 %
^GSPC = 6.544 %
Expected Annual Returns = 13.632 %
CAPM Expected Annual Returns = 22.891 %
Sharpe Ratio = 2.632
